In [2]:
#%cd /aiffel/aiffel/aiffelthon/skgpt-2/

/aiffel/aiffel/aiffelthon/skgpt-2


In [3]:
#!git clone https://github.com/seujung/KoGPT2-summarization.git

Cloning into 'KoGPT2-summarization'...
remote: Enumerating objects: 65, done.
remote: Counting objects: 100% (65/65), done.
remote: Compressing objects: 100% (45/45), done.
remote: Total 65 (delta 33), reused 46 (delta 18), pack-reused 0
Unpacking objects: 100% (65/65), 16.92 KiB | 559.00 KiB/s, done.


In [33]:
%cd /aiffel/aiffel/aiffelthon/skgpt-2/KoGPT2-summarization/

/aiffel/aiffel/aiffelthon/skgpt-2/KoGPT2-summarization


In [34]:
!ls

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
dataset.py	       __pycache__	     run_train.sh	utils.py
generate_sample.ipynb  README.md	     soft_embedding.py
KoBART-summarization   requirements.txt      train_ptuning.py
LICENSE		       run_ptuning_train.sh  train.py


In [7]:
# %cd /aiffel/aiffel/aiffelthon/skgpt-2-lora/

/aiffel/aiffel/aiffelthon/skgpt-2-lora


In [8]:
# !git clone https://github.com/seujung/KoGPT2-LoRA-summarization.git

Cloning into 'KoGPT2-LoRA-summarization'...
remote: Enumerating objects: 11, done.
remote: Counting objects: 100% (11/11), done.
remote: Compressing objects: 100% (9/9), done.
remote: Total 11 (delta 1), reused 6 (delta 0), pack-reused 0
Unpacking objects: 100% (11/11), 8.99 KiB | 3.00 MiB/s, done.


In [3]:
#!pip install transformers==4.5.1
!pip install transformers
#!pip install pytorch-lightning==1.1.0
#!pip install streamlit==0.72.0

In [42]:
!pip install rouge_score

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
  Preparing metadata (setup.py) ... done
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24955 sha256=068861fb8430dcc5f142665148c04bd3a1b8da49ed7803337503c4ff375a115c
  Stored in directory: /aiffel/.cache/pip/wheels/9b/3d/39/09558097d3119ca0a4d462df68f22c6f3c1b345ac63a09b86e
Successfully built rouge-score


In [5]:
import torch
import transformers
import pytorch_lightning
import streamlit
print(torch.__version__)
print(transformers.__version__)
print(pytorch_lightning.__version__)
print(transformers.__version__)

1.9.1+cu111
4.11.3
1.1.0
4.11.3


In [13]:
# pip install torch==1.8.1+cu111 torchvision==0.9.1+cu111 torchaudio==0.8.1 -f https://download.pytorch.org/whl/torch_stable.html

Looking in links: https://download.pytorch.org/whl/torch_stable.html
     |████████████████████████████████| 1982.2 MB 1.3 kB/s              
     |████████████████████████████████| 17.6 MB 44 kB/s              
     |████████████████████████████████| 1.9 MB 6.2 MB/s            
  Attempting uninstall: torch
    Found existing installation: torch 1.9.1+cu111
    Uninstalling torch-1.9.1+cu111:
      Successfully uninstalled torch-1.9.1+cu111
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.10.1+cu111
    Uninstalling torchvision-0.10.1+cu111:
      Successfully uninstalled torchvision-0.10.1+cu111
  Attempting uninstall: torchaudio
    Found existing installation: torchaudio 0.9.1
    Uninstalling torchaudio-0.9.1:
      Successfully uninstalled torchaudio-0.9.1
Note: you may need to restart the kernel to use updated packages.


In [3]:
# 라이브러리 불러오기

import argparse
import logging
import os
import numpy as np
import pandas as pd
#import pytorch_lightning as pl
import re
import torch
#from pytorch_lightning import loggers as pl_loggers
from torch.utils.data import DataLoader, Dataset
from transformers import PreTrainedTokenizerFast, GPT2LMHeadModel
from transformers import GPT2TokenizerFast
from transformers import AutoTokenizer 
from dataset import KoGPTSummaryDataset
from transformers import Trainer, TrainingArguments
from transformers.optimization import AdamW, get_cosine_schedule_with_warmup

In [5]:
# torch cash 초기화
import gc
gc.collect()
torch.cuda.empty_cache()

In [4]:
# 데이터 불러오기

train_df  = pd.read_csv('/aiffel/aiffel/aiffelthon/csv/train_total.csv')
val_df = pd.read_csv('/aiffel/aiffel/aiffelthon/csv/val_total.csv')

print(type(train_df), type(val_df))
print(len(train_df), len(val_df))

<class 'pandas.core.frame.DataFrame'> <class 'pandas.core.frame.DataFrame'>
279992 35004


In [5]:
# drop : Id, Category

train_df.drop(columns = ['Id','Category'], axis = 1, inplace = True)
val_df.drop(columns = ['Id','Category'], axis = 1, inplace = True)

In [5]:
# 데이터 탐색 - train

train_df.sample(10)

,Text,Summary
173173,"['근데 좋겠따 중간에 자러갔다올수도있고 우리는 그런 공간이없어', '졸리댱ㅠㅠ 나...",이 회사는 중간에 자러 갔다 올 수 있는 공간이 없지만 매일 커피를 마시러 나갈 수...
162108,"['수욜은 오후7시에 퇴근해야해', '왜쥬', '도넛간담회2차', '한번 더 하는겨...",수요일에는 2차 도넛 간담회가 있어서 오후 7시에 퇴근해야 한다고 한다.
242190,"['지하촐 사람 개마눔ㅍ ㅠ 잠실쓰.. 롯데월드 솔플각..?', '갑자기?ㅋㅋ', ...",지하철에 사람이 많고 버스도 오지 않아 삼십분째 기다리고 있다.
195865,"['#@이름#! 파스타 접속해 너 자고있다고! 어쩔거야', '퇴출시켜', '뭐? 저...",파스타 게임에 접속하라고 하고 있다.
96799,"['#@이름# 내일 출근해야대는데 언제 잘거야', '마죠ㅠㅠ 지금이순간!', '지금...",내일 출근을 해야 해서 지금 잠을 잘 것이다.
179179,['아씨 열두시에 토스 만보기 들어갈라했는데 시계 이제 봄... 45분까지 생각하고...,12시에 토스 만보기에 들어가지 못했다고 말했다.
178688,"['혼자 맥 사셨는데 저 맥은 어떻게 되는거지 나 주나 (잿밥관심)', 'ㅋㅋㅋㅋ맥...",혼자 맥 사셨는데 저 맥은 어떻게 되냐고 이야기하고 있다.
227554,"['소파나 침대 옆으로 이동식 간이테이블이 잇었으면 좋겠어', '사이드 테이블을 말...",소파나 침대 옆에 간이 테이블이 있으면 편할 것 같다.
217033,"['아진짜 너무더워 더워죽겠네', '나는 이제 시원해졌어...후', '이따 위원님 ...",날씨가 너무 더워서 위원님이 가시면 에어컨 냉방을 다시 켤 것이다.
70104,"['슬슬저녁먹을시간이네~~~', '이모가게와서감자먹었엉ㅋㅋㅋ', '감자~~~? 찐감...",이모 가게에 와서 찐 감자를 먹었다.


In [13]:
# 데이터 탐색 - val

val_df.sample(10)

,Id,Text,Summary,Category
26797,88a5ef59-b550-580e-bc10-b95636412bcc,"['내 쟈기 뭐야 그걸', '그니까', '번역한 것도 웃겨', '영어로 는 그럴듯한...",영어로는 그럴듯한데 번역한 것이 웃기다.,개인 및 관계
33942,9829e1d4-7d5e-5544-bf49-ed8f7f0232d7,"['진짜 나 어재 머앙먹고 음식 만들고잇엇는데ㅔ머멋냐구 햇을때 억울!!', 'ㅎㅎㅎ...",어제 뭐 안 먹고 음식 만들고 있는데 뭐 먹냐고 해서 억울했다고 하니 달그락 소리 ...,식음료
594,d84cf094-2dc6-5e3c-a7d1-76e776476be9,"['쯧 도서관이 개꿀이여 쉬다가 책만 찾으면돼 #@이모티콘#', '오늘 드뎌쉰다 ㅋ...",도서관에서 일하면 쉬다가 책만 찾으면 된다고 하며 내년 학기 중에 도서관 근로를 하...,일과 직업
23917,97c45880-775d-53f3-8ddf-b81e2da4b875,"['냄새나는거', '복도까지 다 들림', '먹지말라도 해놓고', '그니께...안된다...",아줌마들이 음식을 먹어서 냄새와 소리가 난다.,개인 및 관계
444,99b1d7fe-82e9-5812-950a-ac32ea7a2266,"['아니 나 고민이야', '뭐 말만해', '인수인계 받는 애가 너무 느려서 일주일만...",인수인계받은 애가 너무 느려서 일주일만 더 일해달라고 하는데 더 일하기 싫지만 걱정...,일과 직업
31082,7f2dcb99-9f55-5c30-8464-fb623c4df4c6,"['누나누나 초콜릿 싫어해요?', '아뉘 ㅎㅎㅎ 나 초콜릿 좋아하는뎅 아 너가준거 ...",초콜릿 좋아하는데 살 뺀다고 안 먹었다고 하자 하리보는 잘 먹지 않았냐고 하니 듣고...,개인 및 관계
3104,9e6586be-9505-5838-a49a-20da9bf8ff92,"['일단 그럼 투어를 해보는게 나을지도 모르겠다 어렵다 ㅋㅋ큐ㅠ', '우리가 렌탈하...",렌털을 하는 게 낫다고 생각하며 사진이 중요하긴 하다고 생각한다.,주거와 생활
24152,cb6d45dd-1b4f-5b84-b124-df2b45f68a16,"['요즘은 욕이지', 'ㅋㅋㅋㅋㅋㅋ돈을 누가 정하냐', 'ㅋㅋㅋ요즘 추세가 다 오여...",결혼식에 다녀오기 위해서 서울에서 대전까지 다녀왔다.,개인 및 관계
3820,5f999500-db26-54d9-850f-2f417772ed2e,"['아니 내일 뭐입늘거에여?', '움,, 춥겠디?̊̈ 셔츠입울래', '아 춥겠다 진...",추운 날씨를 고려해 내일 입을 옷을 결정하고 있다.,주거와 생활
20357,0521646b-e9ce-5491-879a-98dfdd416ed3,['나오늘뿌염하러미용실갔는디 내옆머리잘ㅇ라놨어 아진짜 나울뻔 앞머리진짜상태대박이여'...,뿌리염색(뿌염)하러 미용실에 갔다가 앞머리가 생각보다 많이 잘렸다.,미용과 건강


In [15]:
train[train['Text'].str.contains(r'[#@]+[가-힣A-Za-z#]+')].sample(10)

,Id,Text,Summary,Category
260509,73a4bfa8-1004-57cb-bd22-fee36f28e699,['#@이모티콘# 언니 이번주에 올림픽공원 갈래요?? #@이름# 미용해서 더 이뻐요...,이번주 목요일에 점심을 먹고 올림픽공원을 가기로 했다.,행사
10755,c501d27f-0b8c-5e70-bfff-84507ba4340d,['오빠 나오늘 백화점가서 모던하우스 구경햇거든 잠시 거기 너무 귀여운게 잇는거야ㅜ...,오늘 백화점에 가서 모던 하우스를 잠깐 구경했는데 손에 안 묻는 크레파스와 공룡 인...,상거래(쇼핑)
34008,ea47a37c-d9c0-57b7-a256-c17cd0904da7,"['어플이야? 쌩인줄 암튼 괜차나!!!!!', '하 루비레이져 하고 그 우울한 마...",루비레이저 시술을 하고 우울한 마음이지만 눈 화장하니까 괜찮아 보여서 다음 주에 놀...,개인 및 관계
273074,8d9c94cf-51f3-5470-bb80-a34b37d8033d,"['누구는 시청광장이라 하구 광화문이겠지', '광화문 아녀,,??', '#@시스템#...",단체 시위가 시청광장에서 열리는지 광화문에서 열리는지에 대해 말한다.,행사
249984,50a813de-06db-5e4e-bf57-4ee8de18a55b,"['마전옴?', '#@시스템#사진# 마전행 놓치고 대전복합으로 왔다ㅎㅎ 나 오늘 수...",지하철을 30분을 기다렸는데 안 와서 포기하고 지금은 대전복합 터미널에 있다.,주거와 생활
57049,48a94050-5061-5747-a85e-a0b7dc53dafe,"['아 아 나 전현무지', '잊지마', '순간 잊었어... 반성할게 #@이모티콘#'...",전현무와 김호중을 닮았다고 하니 김호중보다 전현무가 낫다고 한다.,개인 및 관계
156200,31c3ab8e-bdfc-5546-ae10-46f88b843e94,"['#@시스템#사진#', '힛 살좀빠졌니?', '안재봣는딩 #@이모티콘#', '얼굴...",상대방이 보낸 사진을 보고 얼굴이 갸름해졌다며 살이 빠졌냐고 물어보고 있다.,미용과 건강
244695,1b998847-99da-5c31-b4c1-279871ae95df,"['11월 대면 바로 입겠쥐 ?패딩', '추운날 바로입자', 'ㅠㅠ', '왜냐면 오...",11월이 되면 패딩을 입을까 물으니 추운 날 바로 입자고 한다.,주거와 생활
169735,3cb1eb1e-1b34-56ac-a5b4-f2beab05975b,"['#@이름# 오픈하고 심심하면 이것 좀 추가해조(~˘▽˘)~♡', '아쥔짜 성가시...",오픈하고 시간이 남으면 노래를 추가해달라고 부탁에 이미 추가된 곡이 많다고 말하고 있다.,일과 직업
73607,1e77a51e-b2cf-5209-9442-e3a47ffa2ec6,['네^^ 밥이 다 지어지기 4분 전 방문햇다고 저나가 오더군요^^ 울집에서 얘기해...,밥을 하는 동안에 에스케이(SK)를 설치를 하고 있다.,개인 및 관계


In [6]:
# 데이터 전처리

def preprocess_sentence(sentence):
    sentence = sentence.lower() # 텍스트 소문자화
    sentence = re.sub(r'\([^)]*\)', '', sentence) # 괄호로 닫힌 문자열 (.) 제거
    sentence = re.sub(r'[#@]+[가-힣A-Za-z#]+', ' ', sentence)
    sentence = re.sub(r'[ㄱ-ㅎㅏ-ㅣ]+[/ㄱ-ㅎㅏ-ㅣ]', '', sentence) # 여러개 자음과 모음을 삭제한다.
    sentence = re.sub("[^가-힣a-z0-9-.,!?]", " ", sentence) # 지정한 문자 제외 공백으로 전환
    sentence = re.sub(r'[" "]+', " ", sentence) # 여러개 공백을 하나의 공백으로 바꿉니다.
    sentence = sentence.strip() # 문장 양쪽 공백 제거

    return sentence

In [22]:
sen = '#@시스템#사진# 아 이거 뭐고 ㅋㅋㅋㅋㅋ ㅏㅏㅏㅏㅏ(존웃 ㅅㅂ)'
preprocess_sentence(sen)

'아 이거 뭐고'

In [7]:
# 전체 Text 데이터에 대한 전처리 (1)
from tqdm import tqdm
clean_text = []

for s in tqdm(train_df['Text']):
    clean_text.append(preprocess_sentence(s))
    
train_df['Text'] = clean_text

100%|██████████| 279992/279992 [00:10<00:00, 27300.75it/s]


In [8]:
# 전체 headlines 데이터에 대한 전처리 
clean_headlines = []

for s in tqdm(train_df['Summary']):
      clean_headlines.append(preprocess_sentence(s))
        
train_df['Summary'] = clean_headlines

100%|██████████| 279992/279992 [00:03<00:00, 76776.52it/s]


In [9]:
# 전체 Text 데이터에 대한 전처리 (1)

clean_text_val = []

for s in tqdm(val_df['Text']):
    clean_text_val.append(preprocess_sentence(s))
    
val_df['Text'] = clean_text_val

100%|██████████| 35004/35004 [00:01<00:00, 27534.48it/s]


In [10]:
# 전체 headlines 데이터에 대한 전처리 
clean_headlines_val = []

for s in tqdm(val_df['Summary']):
      clean_headlines_val.append(preprocess_sentence(s))
        
val_df['Summary'] = clean_headlines_val

100%|██████████| 35004/35004 [00:00<00:00, 75388.56it/s]


In [33]:
train_df.head()

,Text,Summary
0,"그럼 날짜는 가격 큰 변동 없으면 6.28-7.13로 확정할까? , 우리 비행포함 ...","비행기 표 가격에 대해 이야기하며, 특가 이벤트를 기다리고 있다."
1,"kf마스크만 5부제 하는거지? , 응. 면마스크는 아무때나 사도될껀? , 면마스크말...",비염이 있어서 싸게 나온 일회용 부직포 마스크를 사두려고 한다.
2,"아 근데 케이크 업체들 봤는데 중앙동쪽 거기는 맛만있고 디자인은 그냥그런것같애 , ...",케이크 업체 중 중앙동 쪽은 맛만 있고 디자인은 별로고 고잔동 케이크 업체는 배달도...
3,"칫솔사야하는데 쓱으로 살까? , 뭘 칫솔사는것까지 물어보시남 , 아 그 왕칫솔 또 ...",칫솔을 3개월에 하나씩 바꿔서 왕 칫솔 사러 신세계 가자고 했다.
4,"잠도안오네 얼릉 고구마츄 먹고싶단 , 그게 그렇게 맛있었어??? 아주 여보 빼이보릿...",잠도 안 와서 고구마 말랭이를 양심상 하나만 먹으려고 한다.


In [24]:
train_df['Text'].iloc[0]

'그럼 날짜는 가격 큰 변동 없으면 6.28-7.13로 확정할까? 우리 비행포함 15일이야?? 16일! 아 너 나한테 돈 보내주면 지금 할 수 잇옹 얼마야 최종 결제액이? 잠시만 인당 952 900 합쳐서 1 905 800 근데 나중에 특가 뜰 수도 있으려나..? 좀 더 두고볼까? 뜨기야 뜨겠지 웅웅 보니까 아시아나는 특가 이벤트 꽤 하는 것 같아서 일단 두고보장 그래 구럼 일단 자자'

In [11]:
# DF > data Set으로 전환

from datasets import Dataset
train_data = Dataset.from_pandas(train_df) 
val_data = Dataset.from_pandas(val_df.iloc[::2, :])
test_data = Dataset.from_pandas(val_df.iloc[1::2, :])

In [12]:
print(train_data)
print(val_data)
print(test_data)

Dataset({
    features: ['Text', 'Summary'],
    num_rows: 279992
})
Dataset({
    features: ['Text', 'Summary'],
    num_rows: 17502
})
Dataset({
    features: ['Text', 'Summary'],
    num_rows: 17502
})


### Tokenizer

In [13]:
# Model 체크포인트 만들기

model_checkpoint = 'skt/kogpt2-base-v2'

In [76]:
from transformers import PreTrainedTokenizerFast

tokenizer = GPT2TokenizerFast.from_pretrained("skt/kogpt2-base-v2",
  bos_token='</s>', eos_token='</s>', unk_token='<unk>',
  pad_token='<pad>', mask_token='<mask>')

loading file https://huggingface.co/skt/kogpt2-base-v2/resolve/main/vocab.json from cache at None
loading file https://huggingface.co/skt/kogpt2-base-v2/resolve/main/merges.txt from cache at None
loading file https://huggingface.co/skt/kogpt2-base-v2/resolve/main/tokenizer.json from cache at /aiffel/.cache/huggingface/transformers/fd8418e6675550cbca8ad6c102d717aa89372eb7a632ad3168300c7fed43491c.db074bfdd88bec54455de5ee2400efdbc64d4acf449a44d5f314e79c1eadc611
loading file https://huggingface.co/skt/kogpt2-base-v2/resolve/main/added_tokens.json from cache at None
loading file https://huggingface.co/skt/kogpt2-base-v2/resolve/main/special_tokens_map.json from cache at None
loading file https://huggingface.co/skt/kogpt2-base-v2/resolve/main/tokenizer_config.json from cache at None
loading configuration file https://huggingface.co/skt/kogpt2-base-v2/resolve/main/config.json from cache at /aiffel/.cache/huggingface/transformers/13bb826cf24517d7849a701e02452715a67c5e560142be3d4735442b2a545809

In [14]:
tokenizer = AutoTokenizer.from_pretrained("skt/kogpt2-base-v2",
  bos_token='</s>', eos_token='</s>', unk_token='<unk>',
  pad_token='<pad>', mask_token='<mask>')

Downloading:   0%|          | 0.00/1.00k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.83M [00:00<?, ?B/s]

In [70]:
tokenizer.tokenize("안녕하세요. 한국어 GPT-2 입니다.😤:)l^o")

['▁안녕',
 '하',
 '세',
 '요.',
 '▁한국어',
 '▁G',
 'P',
 'T',
 '-2',
 '▁입',
 '니다.',
 '😤',
 ':)',
 'l^o']

In [15]:
def transform(batch):
    train_encode = tokenizer(
      text = batch['Text'],
      text_target = batch['Summary'],
      truncation = True,
      padding = 'max_length',
      max_length=128)
    train_encode['labels'] = train_encode['input_ids'].copy()
    return train_encode

In [74]:
!pip uninstall -y transformers

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Found existing installation: transformers 4.11.3
Uninstalling transformers-4.11.3:
  Successfully uninstalled transformers-4.11.3


In [75]:
!pip install transformers

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
     |████████████████████████████████| 5.5 MB 6.7 MB/s            
     |████████████████████████████████| 182 kB 82.9 MB/s            
     |████████████████████████████████| 7.6 MB 56.4 MB/s            
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.10.3
    Uninstalling tokenizers-0.10.3:
      Successfully uninstalled tokenizers-0.10.3
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.0.19
    Uninstalling huggingface-hub-0.0.19:
      Successfully uninstalled huggingface-hub-0.0.19
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. T

In [16]:
train_data = train_data.map(transform)

  0%|          | 0/279992 [00:00<?, ?ex/s]

In [17]:
train_data

Dataset({
    features: ['Text', 'Summary', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 279992
})

In [18]:
def transform_val(batch):
    val_encode =  tokenizer(
      text = batch['Text'],
      text_target = batch['Summary'],
      truncation = True,
      padding = 'max_length',
      max_length=32)
    val_encode['labels'] = val_encode['input_ids'].copy()
    return val_encode

In [19]:
val_data = val_data.map(transform_val)

  0%|          | 0/17502 [00:00<?, ?ex/s]

In [21]:
import datasets
from math import log
from numpy import array
from numpy import argmax

rouge = datasets.load_metric("rouge")

# def beam_search_decoder(data, k):
#     sequences = [[list(), 0.0]]
#     # walk over each step in sequence
#     for row in data:
#         all_candidates = list()
#         # expand each current candidate
#         for i in range(len(sequences)):
#             seq, score = sequences[i]
#             for j in range(len(row)):
#                 candidate = [seq + [j], score - log(row[j])]
#                 all_candidates.append(candidate)
#         # order all candidates by score
#         ordered = sorted(all_candidates, key=lambda tup:tup[1])
#         # select k best
#         sequences = ordered[:k]
#     return sequences


def compute_metrics(pred):
    labels_ids = pred.label_ids
    pred_ids = pred.predictions

    # np.where(a < 0)

    # print(labels_ids.shape, pred_ids.shape, type(labels_ids), type(pred_ids), sep = '\n\n')

    # all unnecessary tokens are removed
    
    # beam search
    # result = []
    # for i in range(pred_ids.shape[0]):
    #     result.append(beam_search_decoder(pred_ids[i], 3))

    # print(result)

    pred_str = tokenizer.batch_decode(np.argmax(pred_ids, axis =2), skip_special_tokens=True)




    # labels_ids[labels_ids == -100] = tokenizer.pad_token_id
    label_str = tokenizer_gpt3.batch_decode(labels_ids, skip_special_tokens=True)

    rouge_output = rouge.compute(predictions=pred_str, references=label_str, rouge_types=["rouge2"])["rouge2"].mid

    return {
        "rouge2_precision": round(rouge_output.precision, 4),
        "rouge2_recall": round(rouge_output.recall, 4),
        "rouge2_fmeasure": round(rouge_output.fmeasure, 4),}

In [22]:
model = GPT2LMHeadModel.from_pretrained(model_checkpoint)

Downloading:   0%|          | 0.00/513M [00:00<?, ?B/s]

In [56]:
from transformers import DataCollatorForLanguageModeling

In [23]:
from transformers import DataCollatorWithPadding

In [24]:
data_collator = DataCollatorWithPadding(tokenizer = tokenizer)

In [25]:
training_args = TrainingArguments(
    output_dir="/aiffel/aiffel/aiffelthon/ch3000/",
    #num_train_epochs=1,  # demo
    max_steps = 3000,
    do_train=True,
    do_eval=True,
    per_device_train_batch_size=8,  # demo
    per_device_eval_batch_size=8,
    learning_rate=3e-05,
    save_steps= 1000,
    eval_steps= 1000,
    warmup_steps=1000,
    #weight_decay=0.1,
    #label_smoothing_factor=0.1,
    #predict_with_generate=True,
    #logging_dir="/aiffel/aiffel/aiffelthon/logs",
    #logging_steps=8000,
    save_total_limit=3,
    fp16 = True)

In [26]:
trainer = Trainer(
    model=model,                 # 학습시킬 model
    args=training_args,          # TrainingArguments을 통해 설정한 arguments
    train_dataset=train_data,    # training dataset
    eval_dataset=val_data,       # evaluation dataset
    data_collator = data_collator,
    compute_metrics=compute_metrics,
)

max_steps is given, it will override any value given in num_train_epochs
Using cuda_amp half precision backend


In [27]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `GPT2LMHeadModel.forward` and have been ignored: Text, Summary. If Text, Summary are not expected by `GPT2LMHeadModel.forward`,  you can safely ignore this message.
/opt/conda/lib/python3.9/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 279992
  Num Epochs = 1
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 3000
  Number of trainable parameters = 125164032
You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encod

Step,Training Loss
500,2.976100
1000,1.110700
1500,1.112400
2000,1.105000
2500,1.090700
3000,1.078100


Saving model checkpoint to /aiffel/aiffel/aiffelthon/ch3000/checkpoint-1000
Configuration saved in /aiffel/aiffel/aiffelthon/ch3000/checkpoint-1000/config.json
Model weights saved in /aiffel/aiffel/aiffelthon/ch3000/checkpoint-1000/pytorch_model.bin
Saving model checkpoint to /aiffel/aiffel/aiffelthon/ch3000/checkpoint-2000
Configuration saved in /aiffel/aiffel/aiffelthon/ch3000/checkpoint-2000/config.json
Model weights saved in /aiffel/aiffel/aiffelthon/ch3000/checkpoint-2000/pytorch_model.bin
Saving model checkpoint to /aiffel/aiffel/aiffelthon/ch3000/checkpoint-3000
Configuration saved in /aiffel/aiffel/aiffelthon/ch3000/checkpoint-3000/config.json
Model weights saved in /aiffel/aiffel/aiffelthon/ch3000/checkpoint-3000/pytorch_model.bin


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=3000, training_loss=1.4121619669596355, metrics={'train_runtime': 515.7014, 'train_samples_per_second': 46.539, 'train_steps_per_second': 5.817, 'total_flos': 1567752192000000.0, 'train_loss': 1.4121619669596355, 'epoch': 0.09})

In [36]:
! pip install pytorch-lightning==1.1.0
! pip install streamlit==0.72.0

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
     |████████████████████████████████| 665 kB 7.1 MB/s            
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
     |████████████████████████████████| 7.4 MB 5.9 MB/s            
     |████████████████████████████████| 78 kB 10.2 MB/s            
     |████████████████████████████████| 812 kB 70.0 MB/s            
     |████████████████████████████████| 182 kB 78.2 MB/s            


     |████████████████████████████████| 4.7 MB 65.7 MB/s            
     |████████████████████████████████| 55 kB 7.5 MB/s             
     |████████████████████████████████| 63 kB 3.0 MB/s             
     |████████████████████████████████| 338 kB 66.7 MB/s            


In [35]:
import torch.nn.functional as F
from train_ptuning import KoGPTConditionalGeneration
from utils import generate_next_token

ModuleNotFoundError: No module named 'pytorch_lightning'

In [42]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model = KoGPTConditionalGeneration.load_from_checkpoint('~/aiffel/aiffel/aiffelthon/ch3000/checkpoint-3000/', hparams=hparams)

Could not locate the tokenizer configuration file, will try to use the model config instead.
loading configuration file config.json from cache at /aiffel/.cache/huggingface/hub/models--skt--kogpt2-base-v2/snapshots/d0c0df48bf2b2c9350dd855021a5b216f560c0c7/config.json
Model config GPT2Config {
  "_name_or_path": "skt/kogpt2-base-v2",
  "_num_labels": 1,
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "author": "Heewon Jeon(madjakarta@gmail.com)",
  "bos_token_id": 0,
  "created_date": "2021-04-28",
  "embd_pdrop": 0.1,
  "eos_token_id": 1,
  "gradient_checkpointing": false,
  "id2label": {
    "0": "LABEL_0"
  },
  "initializer_range": 0.02,
  "label2id": {
    "LABEL_0": 0
  },
  "layer_norm_epsilon": 1e-05,
  "license": "CC-BY-NC-SA 4.0",
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 12,
  "n_positions": 1024,
  "pad_token_id": 3,
  "reorder_and_upcast_attn": false,


NameError: name 'hparams' is not defined

In [41]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model = KoGPTConditionalGeneration.load_from_checkpoint('~/aiffel/aiffel/aiffelthon/ch3000/checkpoint-3000/', hparams=hparams)
batch_size = 16

# map data correctly
def generate_summary(batch):
    # Tokenizer will automatically set [BOS] <text> [EOS]
    inputs = tokenizer(batch["Text"], padding="max_length", truncation=True, max_length=32, return_tensors="pt")
    input_ids = inputs.input_ids.to("cuda")
    attention_mask = inputs.attention_mask.to("cuda")
    
    outputs = model.generate(input_ids, attention_mask=attention_mask)
    
    # all special tokens including will be removed
    output_str = tokenizer.batch_decode(outputs, skip_special_tokens=True)

    batch["pred"] = output_str

    return batch
results = test_data.map(generate_summary, batched=True, batch_size=batch_size, remove_columns=["Text"])
pred_str = results["pred"]
label_str = results["Summary"]

Could not locate the tokenizer configuration file, will try to use the model config instead.
loading configuration file config.json from cache at /aiffel/.cache/huggingface/hub/models--skt--kogpt2-base-v2/snapshots/d0c0df48bf2b2c9350dd855021a5b216f560c0c7/config.json
Model config GPT2Config {
  "_name_or_path": "skt/kogpt2-base-v2",
  "_num_labels": 1,
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "author": "Heewon Jeon(madjakarta@gmail.com)",
  "bos_token_id": 0,
  "created_date": "2021-04-28",
  "embd_pdrop": 0.1,
  "eos_token_id": 1,
  "gradient_checkpointing": false,
  "id2label": {
    "0": "LABEL_0"
  },
  "initializer_range": 0.02,
  "label2id": {
    "LABEL_0": 0
  },
  "layer_norm_epsilon": 1e-05,
  "license": "CC-BY-NC-SA 4.0",
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 12,
  "n_positions": 1024,
  "pad_token_id": 3,
  "reorder_and_upcast_attn": false,


FileNotFoundError: [Errno 2] No such file or directory: '/aiffel/aiffel/aiffel/aiffelthon/ch3000/checkpoint-3000'

In [37]:
import yaml
import torch
import torch.nn.functional as F
from train_ptuning import KoGPTConditionalGeneration
from utils import generate_next_token

In [31]:
model = GPT2LMHeadModel.from_pretrained('/aiffel/aiffel/aiffelthon/ch3000/checkpoint-3000/')
text = '너는 잘가라...'
input_ids = tokenizer.encode(text, return_tensors='pt')
gen_ids = model.generate(input_ids,
                          max_length=128,
                          repetition_penalty=2.0,
                          pad_token_id=tokenizer.pad_token_id,
                          eos_token_id=tokenizer.eos_token_id,
                          bos_token_id=tokenizer.bos_token_id,
                          use_cache=True)
generated = tokenizer.decode(gen_ids[0])
print(generated)

loading configuration file /aiffel/aiffel/aiffelthon/ch3000/checkpoint-3000/config.json
Model config GPT2Config {
  "_name_or_path": "skt/kogpt2-base-v2",
  "_num_labels": 1,
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "author": "Heewon Jeon(madjakarta@gmail.com)",
  "bos_token_id": 0,
  "created_date": "2021-04-28",
  "embd_pdrop": 0.1,
  "eos_token_id": 1,
  "gradient_checkpointing": false,
  "id2label": {
    "0": "LABEL_0"
  },
  "initializer_range": 0.02,
  "label2id": {
    "LABEL_0": 0
  },
  "layer_norm_epsilon": 1e-05,
  "license": "CC-BY-NC-SA 4.0",
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 12,
  "n_positions": 1024,
  "pad_token_id": 3,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_lab

너는 잘가라... 안다고<pad><unk>다. 한다. 있다.한다.하고라고 이야기고<pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>


In [30]:
test_data['Text'][0]

'너는 잘가라....회사.... 선택 잘해.. , 알겠어 많이 힘들구나... 나도 이제 이력서쓰고 영어도 해야해.. , 우리 똥갱애지.... 화이팅.... 첫 회사 잘들어가야한다! , 좋은 시절 다 지났다..'